# Vehicle Routing Problem with Time Windows

## Objectives and Prerequisites
In this notebook, you will learn how to formulate the Vehicle Routing Problem with Time Windows (VRPTW) as a MIP model.
This modeling example is at the advanced level, where we assume that you know Python and the CPLEX Python API and you have advanced knowledge of building mathematical optimization models.

### CPLEX license

In order to run this Jupyter Notebook properly, you must have a IBM® ILOG CPLEX license.

- You can get a free [Community Edition](http://www-01.ibm.com/software/websphere/products/optimization/cplex-studio-community-edition)
 of CPLEX Optimization Studio, with limited solving capabilities in term of problem size.

- Faculty members, research professionals at accredited institutions can get access to an unlimited version of CPLEX through the
 [IBM® Academic Initiative](https://www.ibm.com/academic/technology/data-science).

## Problem Description 
The VRPTW can be described as follows. Given a set of customers, a set of vehicles, and a depot, the VRPTW is to find a set of routes of minimal total length, starting and ending at the depot, such that each customer is visited by exactly one vehicle to satisfy a specific demand. Customer visit has to respect a requested time window. A vehicle can wait in case of early arrival, but late arrival is not allowed. In connection with customer demands, a capacity constraint restricts the load that can be carried by a vehicle. 

## VRPTW Model Formulation
### Sets and Indices
$\text{Vertex}= {(v_{0},...,v_{n})}$: is the set of nodes with a special node $v_{0}$ called the depot

$\text{Arcs}= \{(v_{i},v_{j}) \in Vertex \times Vertex \}$: Set of arcs between nodes.

$G = (V, A)$: A complete directed graph.

### Parameters 

$c_{i,j}$: Cost of going from vertex $i$ to vertex $j$, for all $(v_{i},v_{j}) \in A$.

$t_{i,j}$: Travel time going from vertex $i$ to vertex $j$, for all $(v_{i},v_{j}) \in A$.

$d_{i} \in \mathbb{R}^+$: Demand of the customer $v_{i} \in \text{V}-v_{0}$. 

$serv_{i} \in \mathbb{R}^+$: Service time for the customer $v_{i} \in \text{V}-v_{0}$. 

$[a_{i},b_{i}]$: Time window for the vertex $v_{i} \in \text{V}$.

$U$: Size of the fleet of vehicles

$Q$: Capacity for a vehicle

#### Assumptions
In the following, we make these additional common assumptions: the cost and the travel time matrices are supposed to be equal, nonnegative and to satisfy the triangle inequality. For the sake of simplicity, we also define $d_{0}=0$ and $serv_{0}=0$.

### Decision Variables
$x_{i, j}^u \in \{0, 1\}$: This variable is equal to 1, if the arc $(v_{i},v_{j}) \in A$ is used by vehicle $u$. Otherwise, the decision variable is equal to zero.

$s_{i}^u \in \mathbb{R}^+$: The time when the vehicle $u$ starts serving vertex $v_{i} \in V$. For the depot, $s_{0}^u$ is the departure time of vehicle $u$.  

### Objective Function
- **Minimum cost routes**. Minimize the total cost of the routes. 

\begin{equation}
\text{Min} \quad Z = \sum_{1 \leq u \leq U}\sum_{(v_{i},v_{j}) \in \text{A}}c_{i,j} \cdot x_{i,j}^u

\end{equation}

### Constraints 
- **Customer satisfaction**. Enforces the visit of every customer. Notice that is is allowed to visit a customer more than once, this relaxation is valid since is not optimal to visit a customer more than once. 

\begin{equation}
\sum_{1 \leq u \leq U} \sum_{v_{j} \in V |(v_{i},v_{j}) \in \text{A}} x_{i, j}^u \geq 1 \quad \forall \quad v_{i} \in \text{V}-v_{0}

\end{equation}

- **Route structure**. The following constraints define the route structure for the vehicles.

\begin{equation}
\sum_{v_{j} \in V |(v_{i},v_{j}) \in \text{A}} x_{i,j}^u - \sum_{v_{j} \in V |(v_{i},v_{j}) \in \text{A}} x_{j,i}^u  = 0 \quad \forall \quad  v_{i} \in V, 1 \leq u \leq U

\end{equation}

\begin{equation}

\sum_{v_{j} \in V |(v_{i},v_{j}) \in \text{A}} x_{0,i}^u \leq 1 \quad \forall \quad 1 \leq u \leq U

\end{equation}

- **Capacity**. The total demand satisfied by a vehicle with a route cant be greater than teh vehicle capacity.

\begin{equation}
\sum_{(v_{i},v_{j}) \in \text{A}} d_{i} \cdot x_{i,j}^u \leq Q \quad \forall \quad 1 \leq u \leq U

\end{equation}

- **Time windows**. The following constraints concern time windows satisfaction.

\begin{equation}

s_{i}^u + serv_{i} + t_{i,j} - s_{j}^u + M \cdot x_{i,j}^u \leq M \quad \forall \quad (v_{i},v_{j})\in A, v_{i}\neq v_{0},1 \leq u \leq U

\end{equation}

\begin{equation}

s_{i}^u + serv_{i} + t_{i,0} - b_{0} + M \cdot x_{i,0}^u \leq M \quad \forall \quad (v_{i},v_{j})\in A,1 \leq u \leq U

\end{equation}

\begin{equation}

a_{i} \leq s_{i}^u \leq b_{i} \quad \forall \quad v_{i} \in V,1 \leq u \leq U

\end{equation}


## Python implementation

Consider a petroleum company in Lima, Peru that need to plan the routes for their vehicles to supply gas from one refinery to some of its gas stations across the town. 

This modeling example requires to import the following libraries.
* **docplex** calls IBM® ILOG CPLEX algorithms to solve MIP models.
* **math** access to mathematical functions.
* **itertools** implements a number of iterator building blocks.
* **folium** creates maps.

### Model Definition 
For the model definition, a new class is created to represent a node, this way,all of the node properties (index, demand, service time and time window) are contained in a single object.


In [1]:
class Node:
    def __init__(self, index, demand, serv_time, time_window, lat, long):
        self.index = index
        self.demand = demand
        self.serv_time = serv_time
        self.time_window = time_window
        self.lat = lat
        self.long = long

    def __index__(self):
        return self.index

    def __hash__(self):
        return self.index

    def __repr__(self):
        return f"Customer <{self.index}>"

In [2]:
from docplex.mp.model import Model

def build_VRPTW_problem(capacity, nodes, times, vehicles,**kwargs):
    
    #Parameters
    customers = [cstmrs for cstmrs in nodes if cstmrs.index != 0] 
    #Big M
    bigM = 1000

    #Model declaration
    m = Model(name = 'VRPTW', **kwargs)

    #----- Variables -----
    #Arc covered by vehicle
    m.x_var = m.binary_var_cube(vehicles,nodes,nodes, name= 'route')
    #Start time for the service of a customer
    m.s_var = m.continuous_var_matrix(vehicles, nodes, name = 'service_time')

    #----- Constraints -----
    #Customer satisfaction
    m.covering_cts = []
    for i in customers: 
        cust_satisf_ct = m.sum(
            m.x_var[u,i,j] for j in nodes for u in vehicles)>= 1
        cust_satisf_ct.name = 'customer_satisf_{0!s}'.format(i.index)
        m.covering_cts.append(cust_satisf_ct)

    m.add_constraints(m.covering_cts)
    
    #Route structure
    m.flow_cts = []
    for i in nodes:
        for u in vehicles:
            flow_ct = (m.sum(m.x_var[u,i,j] for j in nodes) - 
                m.sum(m.x_var[u,j,i] for j in nodes)) == 0
            flow_ct.name = 'flow_constraint_{0!s}_{1!s}'.format(i.index,u)
            m.flow_cts.append(flow_ct)

    for u in vehicles: 
        flow_ct = m.sum(m.x_var[u,nodes[0],i] for i in nodes) <= 1
        flow_ct.name = 'flow_constraint_{0!s}_{1!s}_origin'.format(nodes[0].index,u)
        m.flow_cts.append(flow_ct)

    m.add_constraints(m.flow_cts)

    #Vehicle capacity
    m.capacity_cts = []
    for u in vehicles:
        capacity_ct = m.sum(nodes[i].demand * m.x_var[u,i,j] for i in nodes for j in nodes) <= capacity
        capacity_ct.name = 'capacity_vehicle{0!s}'.format(u)
        m.capacity_cts.append(capacity_ct)
    
    m.add_constraints(m.capacity_cts)

    #Time windows
    m.time_window_cts = []
    for i in nodes:
        for j in customers: 
            for u in vehicles:
                time_window_ct = m.s_var[u,i] + i.serv_time + times[i.index][j.index] - m.s_var[u,j] + bigM*m.x_var[u,i,j] <= bigM
                time_window_ct.name = 'time_window_{0!s}_{1!s}_{2!s}'.format(u,i.index,j.index)
                m.time_window_cts.append(time_window_ct)
    
    for i in customers:
        for u in vehicles: 
            time_window_ct = m.s_var[u,i] + i.serv_time + times[i][0] - nodes[0].time_window[1] + bigM*m.x_var[u,i,nodes[0]] <= bigM
            time_window_ct.name = 'time_window_{0!s}_{1!s}_{2!s}_arrival'.format(u,i.index,nodes[0].index)
            m.time_window_cts.append(time_window_ct)

    for i in nodes:
        for u in vehicles:
            time_window_ct_lb = i.time_window[0] <= m.s_var[u,i]
            time_window_ct_lb.name = 'time_window_lb_{0!s}'.format(i.index)
            time_window_ct_ub = i.time_window[1] >= m.s_var[u,i]
            time_window_ct_ub.name = 'time_window_ub_{0!s}'.format(i.index)
            m.time_window_cts.append(time_window_ct_lb)
            m.time_window_cts.append(time_window_ct_ub)

    m.add_constraints(m.time_window_cts)

    #----- Objective Function -----
    #Minimize the total cost
    m.total_cost = m.sum(times[i][j]*m.x_var[u,i,j] for i in nodes for j in nodes for u in vehicles)
    m.minimize(m.total_cost)

    return m


### Problem data
The problem we will adress for this example is to generate the minimum cost routes for a petroleum company located in Lima, Peru. The company has one refinery, four gas stations and two vehicles to supply all the gas stations. Each one of the gas stations has a known demand and a specific time window to be served.

In [3]:
'''
To create a node we use the following parameters
index: int
    The index of the node in the directed graph for the network (The index 0 is used for the refinery)
demand: float
    The demand for each one of the nodes 
serv_time: float
    The service time for the nodes
time_window: list
    The time window where the node can be served
lat: float
    latitude of the node
long: float
    longitude of the node
'''
refinery = Node(0,0,0,[0,120], -11.920058,-77.129718)
gas_station_1 = Node(1,35,5,[15,80], -12.050424, -76.961938)
gas_station_2 = Node(2,25,6,[35,115], -12.139093, -77.017575)
gas_station_3 = Node(3,35,5,[12,68], -11.971129, -76.753334)
gas_station_4 = Node(4,45,6,[24,96], -12.056154, -77.057922)

vehicles = [0,1]
capacity = 100

nodes = [refinery, gas_station_1, gas_station_2, gas_station_3, gas_station_4]

Now we would like to visualize the situation previous to the optimization.


In [3]:
import folium
from folium import Marker

m = folium.Map(location=[-12.047209, -77.069313], tiles='openstreetmap', zoom_start=11)
for node in [n for n in nodes if n.index != 0]:
    Marker(location= [node.lat, node.long], icon= (folium.Icon(color='gray', icon='tint', prefix='fa'))).add_child(folium.Popup('gas station {}'.format(node.index))).add_to(m)

Marker(location= [nodes[0].lat,nodes[0].long], icon= (folium.Icon(color='red', icon='industry', prefix='fa'))).add_child(folium.Popup('Refinery')).add_to(m)
m

To determine the traveling times between two customers, the euclidean distance is used and multiplied by 100. Another way to do this is to integrate third party API's that allow to calculate the real time between customers taking into account factors like average or actual traffic in the road network. 

In [4]:
times = [[round(((node_origin.lat - node_destin.lat)**2 + 
    (node_origin.long - node_destin.long)**2)**(1/2),3)*100 
    for node_destin in nodes] for node_origin in nodes]

Now, to generate the full model taking into account the menctioned parameters we use the previously defined function `build_VRPTW_problem` with parameters 100 representing the vehicles capacity, nodes calling the previously created list of nodes, times, and a list with two vehicles. 

In [5]:
mod = build_VRPTW_problem(capacity, nodes, times, vehicles)

The model is solved and then the resulting data is stored in a dictionary.

In [6]:
import json
sol = mod.solve()
sol_dict = json.loads(sol.export_as_json_string())
sol_dict['CPLEXSolution']['variables'][0:6]

[{'index': '3', 'name': 'route_0_Customer <0>_Customer <3>', 'value': '1.0'},
 {'index': '7', 'name': 'route_0_Customer <1>_Customer <2>', 'value': '1.0'},
 {'index': '10', 'name': 'route_0_Customer <2>_Customer <0>', 'value': '1.0'},
 {'index': '16', 'name': 'route_0_Customer <3>_Customer <1>', 'value': '1.0'},
 {'index': '29', 'name': 'route_1_Customer <0>_Customer <4>', 'value': '1.0'},
 {'index': '45', 'name': 'route_1_Customer <4>_Customer <0>', 'value': '1.0'}]

In order to plot the resulting routes for each vehicle, the Lima road network is generated. 

In [5]:
import networkx as nx
import osmnx as ox 
ox.config(log_console=True, use_cache=True)

G = ox.graph_from_point(center_point = (-12.047209, -77.069313),dist = 45000, simplify = True, retain_all = False)
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)

After generating the road network, the nodes and coordinates for the solution route are generated.

In [6]:
v0 = [nodes[0], nodes[3], nodes[1],nodes[2],nodes[0]]
v1 = [nodes[0], nodes[4],nodes[0]]

v = [v0, v1]

routes = []

for vehicle in vehicles:
    for track in zip(v[vehicle],v[vehicle][1:] + v[vehicle][:1]):
        start = (track[0].lat, track[0].long)
        end = (track[1].lat, track[1].long)
        start_node = ox.get_nearest_node(G, start) 
        end_node = ox.get_nearest_node(G, end)
        routes.append([vehicle,nx.shortest_path(G, start_node, end_node, weight='travel_time')])

Data wrangling is performed to plot the solution.

In [7]:
import pandas as pd

node_start = {vehicle: [] for vehicle in vehicles}
node_end = {vehicle: [] for vehicle in vehicles}
X_to = {vehicle: [] for vehicle in vehicles}
Y_to = {vehicle: [] for vehicle in vehicles}
X_from = {vehicle: [] for vehicle in vehicles}
Y_from = {vehicle: [] for vehicle in vehicles}
length = {vehicle: [] for vehicle in vehicles}
travel_time = {vehicle: [] for vehicle in vehicles}

for track in routes:
    for u, v in zip(track[1][:-1], track[1][1:]):
        node_start[track[0]].append(u)
        node_end[track[0]].append(v)
        length[track[0]].append(round(G.edges[(u, v, 0)]['length']))
        travel_time[track[0]].append(round(G.edges[(u, v, 0)]['travel_time']))
        X_from[track[0]].append(G.nodes[u]['x'])
        Y_from[track[0]].append(G.nodes[u]['y'])
        X_to[track[0]].append(G.nodes[v]['x'])
        Y_to[track[0]].append(G.nodes[v]['y'])

routes_dfs = {vehicle : pd.DataFrame(list(zip(node_start[vehicle], node_end[vehicle], X_from[vehicle], Y_from[vehicle],
               X_to[vehicle], Y_to[vehicle], length[vehicle], travel_time[vehicle])), 
               columns =['node_start', 'node_end', 'X_from', 'Y_from',  'X_to',
               'Y_to', 'length', 'travel_time']) for vehicle in vehicles}

Finally the maps with the solution are plotted

In [8]:
import plotly_express as px
px.set_mapbox_access_token("pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w")

figures = {vehicle: px.line_mapbox(routes_dfs[vehicle], lon= 'X_from', lat='Y_from',
                                      zoom=10, width=1000, height=800,
                                      mapbox_style='streets') for vehicle in vehicles}

for v_figure in vehicles:
    start = (nodes[0].lat, nodes[0].long)
    end = (nodes[0].lat, nodes[0].long)
    figures[v_figure].add_trace(px.line_mapbox(routes_dfs[v_figure], lon= 'X_from', lat='Y_from').data[0])


#### Vehicle 0

In [9]:
figures[0]

#### Vehicle 1

In [10]:
figures[1]

## References

[1] Feillet, D. A tutorial on column generation and branch-and-price for vehicle routing problems. 4OR-Q J Oper Res 8, 407–424 (2010). https://doi.org/10.1007/s10288-010-0130-z

#### For more information about SmartBP
- [Website](http://www.smart-bp.com)
- [Like Us](https://www.facebook.com/Smartbp-122794631689852/?ref=bookmarks)
- [Follow Us](https://twitter.com/Smart_BP)
- [Connect with Us](https://www.linkedin.com/company/smartbp/?viewAsMember=true)
- [YouTube](https://www.youtube.com/c/SmartBP)


Copyright © 2020 SmartBP 